# Native Bayes Classifier
The Naive Bayes algorithm is a classification method based on Bayes' theorem and the assumption of feature condition independence. Its core principle is to use Bayes theorem to calculate the posterior probability of each category under a given data sample, and select the category with the highest posterior probability as the prediction category of the sample. The naive Bayes algorithm assumes that the features are independent of each other, which simplifies the calculation, but may also affect the accuracy of classification. Because of its simplicity and high learning efficiency, naive Bayes algorithm is still widely used in practical applications, especially in the fields of text classification and spam filtering.




## Load Dataset
Read the training dataset into a dataframe and read the test dataset which will be used to estimate the classification accuracy.

In [40]:
import numpy as np
import math
import pandas as pd

train_set=pd.read_csv('./toy_train.csv',index_col=0)#设置第一列为索引列
test_set=pd.read_csv('./toy_test.csv',index_col=0)
train_set


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,2,7,7,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,2,7,7,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,2,7,7,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,2,7,7,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,2,7,7,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,1,3,2,2,0,2,1,0,1,0,...,1,2,7,7,2,1,0,7,4,2
7596,1,3,2,4,0,7,1,0,1,0,...,2,1,6,6,2,1,0,7,4,0
7597,0,5,2,4,1,5,1,0,0,10,...,2,2,7,7,2,2,4,7,4,4
7598,1,3,2,4,0,2,1,0,1,0,...,1,1,7,6,2,1,0,7,4,4


## Maximum Likelihood Estimation
We first need to calculate the prior probability of the toy classification, i.e：


$$P(\widehat{Y}=y_k)= \frac{\#D\{Y=y_k\}+\varepsilon}{|D|+\varepsilon}  $$



Then we can calculate the conditional probabilities for different toy properties. Assuming that the attributes are not correlated, the conditional probability can be calculated by the following formula:

$$ \widehat{P}(X_j=a | Y=y_k)=\frac{\#D\{X_j=a,Y=y_k\}+\varepsilon}{\#D\{Y=y_k\}+\varepsilon}$$

$$ \widehat{P}(X_1,X_2,...X_M | Y)= \prod^M_{j=1}\widehat{P}(X_j | Y) $$
 where the $\#D\{x\}$ operator denotes the number of elements in the set $D$ that satisfy property $x$ and $ϵ (ϵ = 1)$ is a smoothing factor. $X_i$ is the i-th attribute and $Y$ is the class label. 
 



 Our goal is to train a classifier that will output the probability distribution over possible values of $Y$.  For any example $X$, the probability $P(X) = P(X_1,X_2,···,X_M)$ is constant. We can estimate these parameters using maximum likelihood estimates.



After calculating the conditional probability of each attribute of the toy, the posterior probability of the toy classification under each attribute can be calculated.

In [25]:
### 计算先验概率
def prior_P(data,col_name):
    con1=data[col_name].agg("value_counts") ###按玩具种类计数
    p=(con1+1)/(len(data)+1) ###计算两种玩具种类的先验概率
    return p
    
print(prior_P(train_set,'class'))

class
0    0.520984
1    0.479147
Name: count, dtype: float64



As shown above, the prior probability for a toy of class 0 is $\widehat{P}(Y=0)=0.5209$, and the prior probability for a toy of class 1 is $\widehat{P}(Y=1)=0.4791$



Next we can calculate $\widehat{P}(X_j | Y)$

In [41]:
###Computed conditional probability
def condition_P(data):
    d0=data[data['class']==0]
    d1=data[data['class']==1]
    c=list(data.columns)
    c.remove('class')
    l0={}
    l1={}
    for i in c:
        p0=(d0[i].agg("value_counts")+1)/(len(d0)+1) #Computed conditional probability
        l0[i]=p0.to_dict()
    for i in c:    
        p1=(d1[i].agg("value_counts")+1)/(len(d1)+1) #Computed conditional probability
        l1[i]=p1.to_dict() ###Save it in dict
    return l0, l1

p_x_y0,p_x_y1=condition_P(train_set)
p_x_y0

{'cap-shape': {5: 0.4734848484848485,
  2: 0.3939393939393939,
  0: 0.0845959595959596,
  3: 0.040656565656565655,
  4: 0.008333333333333333},
 'cap-surface': {0: 0.3782828282828283,
  3: 0.37676767676767675,
  2: 0.24545454545454545},
 'cap-color': {4: 0.2881313131313131,
  3: 0.2474747474747475,
  8: 0.16515151515151516,
  2: 0.15782828282828282,
  9: 0.10126262626262626,
  5: 0.013888888888888888,
  0: 0.012373737373737374,
  1: 0.007575757575757576,
  7: 0.004292929292929293,
  6: 0.004292929292929293},
 'bruises': {1: 0.6921717171717172, 0: 0.30808080808080807},
 'odor': {5: 0.797979797979798,
  0: 0.10126262626262626,
  3: 0.10126262626262626},
 'gill-attachment': {1: 0.9800505050505051, 0: 0.020202020202020204},
 'gill-spacing': {0: 0.7262626262626263, 1: 0.273989898989899},
 'gill-size': {0: 0.9272727272727272, 1: 0.07297979797979798},
 'gill-color': {5: 0.2292929292929293,
  10: 0.225,
  7: 0.2058080808080808,
  9: 0.11237373737373738,
  4: 0.08712121212121213,
  2: 0.05479797


Next, the conditional probability multiplies for each sample based on the training data, i.e. :

$$ \widehat{P}(X_1,X_2,...X_M | Y)= \prod^M_{j=1}\widehat{P}(X_j | Y) $$



As shown above, p0 and p1 in the new dataframe correspond to $\widehat{P}(X_1,X_2,... X_M | Y=0)$  and $\widehat{P}(X_1,X_2,... X_M | Y=1)$

## Model Inference
For K categories, we calculate the posterior probability of each category separately for $[X = x1,x2,···,x_M] $ and get
 
$$log\widehat{P}(Y = y_k|X) ∝ log \widehat{P}(Y = y_k)+\sum _{j=1}^M log \widehat{P}(X_j = x_j|Y = y_k), k = 1,2,···,K $$



To facilitate the calculation, we can rewrite the formula according to the properties of logarithms as:

$$log\widehat{P}(Y = y_k|X) ∝ log \widehat{P}(Y = y_k)+ log \prod _{j=1}^M \widehat{P}(X_j = x_j|Y = y_k), k = 1,2,···,K $$


 If the conditional probability P(Xj = xj|Y = yk) that does not appear in the training set is needed in the test set, then the conditional probability under class k should be


 $$P(X_j = ai|Y = y_k) =\frac{1}{\#D\{Y =y_k\}+N_j} $$


In [42]:
def prod_pxiy(train_data,test_data):
    p0,p1=condition_P(train_data)
    data1=test_data.drop('class',axis=1)
    prior=prior_P(train_data,'class')
    ny0=len(train_data.loc[train_data['class']==0])
    ny1=len(train_data.loc[train_data['class']==1])
    c0=[]
    c1=[]
    for i in data1.values:
        c00=[]
        for j in range(len(i)):
            colname=data1.columns[j]
            k=i[j]
            if k in p0[colname]:
                ###Take out the corresponding conditional probability
                c00.append(p0[colname][k])
            else:
                nj=len(data1.loc[data1[colname]==k])
                f=1/(ny0+nj)
                c00.append(f)
        x0=math.log(np.prod(c00))+math.log(prior[1])
        c0.append(x0)
    for i1 in data1.values:
        c11=[]
        for j1 in range(len(i1)):
            colname1=data1.columns[j1]
            k1=i1[j1]
            if k1 in p1[colname1]:
                ### Take out the corresponding conditional probability
                c11.append(p1[colname1][k1])
            else:
                nj1=len(data1.loc[data1[colname1]==k1])
                f1=1/(ny1+nj1)
                c11.append(f1)
        x1=math.log(np.prod(c11))+math.log(prior[1])
        c1.append(x1)    
    new_data=test_data
    new_data['logp0']=c0
    new_data['logp1']=c1
    return new_data



As shown above, logp0 and logp1 in the new dataframe correspond to $log\widehat{P}(Y=0|X)$ and $log\widehat{P}(Y=1 |X)$




### Construct prediction function and accuracy calculation

Next we will compare the two columns to the class with the largest posterior probability $log\widehat{P}(Y =y_k|X)$ and calculate the correct rate.

In [43]:
def pre(data):
    test=pd.DataFrame()
    test['class']=data['class']
    predict=[]
    for i in range(len(data)):
        if data['logp0'][i]>data['logp1'][i]:
            predict.append(0)
        else:
            predict.append(1)
    test['predict']=predict
    k=0
    for i in range(len(data)):
        if test['class'][i]==test['predict'][i]:
            k=k+1
    print(k/len(data)) 
    return test
          


In [53]:
def prod_pxiy(train_data,test_data):
    p0,p1=condition_P(train_data)
    data1=test_data.drop('class',axis=1)
    prior=prior_P(train_data,'class')
    ny0=len(train_data.loc[train_data['class']==0])
    ny1=len(train_data.loc[train_data['class']==1])
    c0=[]
    c1=[]
    for i in data1.values:
        c00=[]
        for j in range(len(i)):
            colname=data1.columns[j]
            k=i[j]
            if k in p0[colname]:
            ###Take out the corresponding conditional probability
                c00.append(p0[colname][k])
            else:
                nj=len(data1.loc[data1[colname]==k])
                f=1/(ny0+nj)
                c00.append(f)
        x0=math.log(np.prod(c00))+math.log(prior[1])
        c0.append(x0)
    for i1 in data1.values:
        c11=[]
        for j1 in range(len(i1)):
            colname1=data1.columns[j1]
            k1=i1[j1]
            if k1 in p1[colname1]:
                ###Take out the corresponding conditional probability
                c11.append(p1[colname1][k1])
            else:
                nj1=len(data1.loc[data1[colname1]==k1])
                f1=1/(ny1+nj1)
                c11.append(f1)
        x1=math.log(np.prod(c11))+math.log(prior[1])
        c1.append(x1)    
    new_data=test_data
    new_data['logp0']=c0
    new_data['logp1']=c1
    return new_data


def pre(data):
    test=pd.DataFrame()
    test['class']=data['class']
    predict=[]
    for i in range(len(data)):
        if data['logp0'][i]>data['logp1'][i]:
            predict.append(0)
        else:
            predict.append(1)
    test['predict']=predict
    k=0
    for i in range(len(data)):
        if test['class'][i]==test['predict'][i]:
            k=k+1
    print(k/len(data)) 
    return test




Test accuracy on the original train data set：

In [45]:
test1=prod_pxiy(train_set,train_set)
pre(test1)

0.9542105263157895


,class,predict
0,1,0
1,0,0
2,0,0
3,1,0
4,0,0
...,...,...
7595,1,1
7596,1,1
7597,0,0
7598,1,1


Test accuracy on the test data set：

In [33]:
test2=prod_pxiy(train_set,test_set)
pre(test2)

0.40076335877862596


,class,predict
0,1,0
1,1,1
2,0,1
3,1,0
4,0,1
...,...,...
519,0,1
520,0,1
521,0,1
522,1,1


It can be seen from the above test that the degree of overfitting of the model is high, and the accuracy of the model is low when used in non-training sets.

In [34]:
test3=prod_pxiy(test_set,train_set)
pre(test3)

0.6630263157894737


,class,predict
0,1,0
1,0,0
2,0,0
3,1,0
4,0,1
...,...,...
7595,1,1
7596,1,1
7597,0,0
7598,1,1


### Test result interpretation

It can be seen from the above three tests that the model has a high degree of overfitting. Because the training set has a large number of samples and the test set has a small number of samples, when we switch the test set and the training set, the test effect is better than that of the model trained with a large number of data, indicating that the model has obvious overfitting, and the accuracy rate of the model to its own training set is as high as 98%. The effect of the model built with a small amount of data is 20% higher than that of the model built with a large amount of data, indicating that the model is seriously overfitting.

### How to Deal with Overfitting
In machine learning, if a model is too focused on a particular training data and misses the point, then the model is considered overfitting. The answer provided by the model is far from the correct answer, that is, the accuracy is reduced. Such models treat noise in irrelevant data as a signal, which negatively affects accuracy. Even if the model is well trained to lose very little, it still performs poorly on new data.
Cross-validation is a good way to prevent overfitting. In cross-validation, we generate multiple training test partitions and adjust the model. K-fold validation is a standard cross-validation method, that is, the data is divided into K subsets, one of which is used for validation, and the other subsets are used for training the algorithm. We can also remove features and train the model with more relevant data to help identify the signal better, avoiding noise as a signal. Similarly, regularization can be used to reduce the complexity of the model through the penalty-loss function.



### The result after changing the formula
Below are the original calculations



In [52]:
test2

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat,logp0,logp1
0,1,0,2,6,0,2,1,1,1,5,...,5,5,3,1,0,3,0,2,-52.596744,-66.717827
1,1,3,2,1,0,0,1,0,1,0,...,4,3,2,1,0,3,3,2,-47.277545,-46.524802
2,0,4,1,3,1,2,1,0,0,5,...,4,4,2,2,2,3,4,3,-61.157184,-43.011812
3,1,3,1,1,0,0,1,0,1,0,...,3,3,2,1,0,3,3,3,-52.591109,-55.489007
4,0,3,0,2,0,2,1,1,0,4,...,4,4,2,2,2,3,2,1,-53.098211,-44.979096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,0,3,1,3,0,2,0,0,0,6,...,2,2,1,1,2,0,0,2,-75.018319,-73.396161
520,0,4,1,3,0,2,0,0,0,6,...,2,2,0,1,2,0,3,2,-75.766968,-73.417407
521,0,2,1,3,0,2,0,0,0,2,...,2,2,1,1,2,0,0,2,-70.586261,-65.746590
522,1,3,2,3,0,4,1,0,1,0,...,4,4,2,1,0,3,3,2,-53.666263,-29.851144


The function has been re-updated in this report due to a change in the conditional probability formula:
$$ \widehat{P}(X_j=a | Y=y_k)=\frac{\#D\{X_j=a,Y=y_k\}+1}{\#D\{Y=y_k\}+N_j}$$

Since the data used in this task are all class variables starting from 0, the size of $N_j$ can be obtained by using max()+1

In [54]:
###Computed conditional probability
def condition_P(data):
    d0=data[data['class']==0]
    d1=data[data['class']==1]
    c=list(data.columns)
    c.remove('class')
    l0={}
    l1={}
    for i in c:
        #Compute conditional probability
        p0=(d0[i].agg("value_counts")+1)/(len(d0)+max(data[i])+1)
        l0[i]=p0.to_dict()
    for i in c:
        #Compute conditional probability    
        p1=(d1[i].agg("value_counts")+1)/(len(d1)+max(data[i])+1) 
        l1[i]=p1.to_dict() ###Save it in dict
    return l0, l1


def prod_pxiy(train_data,test_data):
    p0,p1=condition_P(train_data)
    data1=test_data.drop('class',axis=1)
    prior=prior_P(train_data,'class')
    ny0=len(train_data.loc[train_data['class']==0])
    ny1=len(train_data.loc[train_data['class']==1])
    c0=[]
    c1=[]
    for i in data1.values:
        c00=[]
        for j in range(len(i)):
            colname=data1.columns[j]
            k=i[j]
            if k in p0[colname]:
            ###Take out the corresponding conditional probability
                c00.append(p0[colname][k])
            else:
                nj=len(data1.loc[data1[colname]==k])
                f=1/(ny0+nj)
                c00.append(f)
        x0=math.log(np.prod(c00))+math.log(prior[1])
        c0.append(x0)
    for i1 in data1.values:
        c11=[]
        for j1 in range(len(i1)):
            colname1=data1.columns[j1]
            k1=i1[j1]
            if k1 in p1[colname1]:
                ###Take out the corresponding conditional probability
                c11.append(p1[colname1][k1])
            else:
                nj1=len(data1.loc[data1[colname1]==k1])
                f1=1/(ny1+nj1)
                c11.append(f1)
        x1=math.log(np.prod(c11))+math.log(prior[1])
        c1.append(x1)    
    new_data=test_data
    new_data['logp0']=c0
    new_data['logp1']=c1
    return new_data


The new results are as follows:

In [55]:
test=prod_pxiy(train_set,test_set)
test

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat,logp0,logp1
0,1,0,2,6,0,2,1,1,1,5,...,5,5,3,1,0,3,0,2,-69.185937,-83.136236
1,1,3,2,1,0,0,1,0,1,0,...,4,3,2,1,0,3,3,2,-63.866739,-62.947051
2,0,4,1,3,1,2,1,0,0,5,...,4,4,2,2,2,3,4,3,-77.745368,-59.437077
3,1,3,1,1,0,0,1,0,1,0,...,3,3,2,1,0,3,3,3,-69.179547,-71.909062
4,0,3,0,2,0,2,1,1,0,4,...,4,4,2,2,2,3,2,1,-69.687152,-61.404362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,0,3,1,3,0,2,0,0,0,6,...,2,2,1,1,2,0,0,2,-91.606503,-89.811006
520,0,4,1,3,0,2,0,0,0,6,...,2,2,0,1,2,0,3,2,-92.355152,-89.832251
521,0,2,1,3,0,2,0,0,0,2,...,2,2,1,1,2,0,0,2,-87.174445,-82.164449
522,1,3,2,3,0,4,1,0,1,0,...,4,4,2,1,0,3,3,2,-70.253440,-46.277781


### Test
Test_set result

In [56]:
pre(test)

0.4026717557251908


,class,predict
0,1,0
1,1,1
2,0,1
3,1,0
4,0,1
...,...,...
519,0,1
520,0,1
521,0,1
522,1,1


And the results didn't change a lot.